# Example of DOV search methods for interpretations (formele stratigrafie)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fsearch_formele_stratigrafie.ipynb)

## Use cases explained below
* Get 'formele stratigrafie' in a bounding box
* Get 'formele stratigrafie' with specific properties
* Get 'formele stratigrafie' in a bounding box based on specific properties
* Select 'formele stratigrafie' in a municipality and return date
* Get 'formele stratigrafie' based on fields not available in the standard output dataframe
* Get 'formele stratigrafie' data, returning fields not available in the standard output dataframe

In [1]:
%matplotlib inline
import inspect, sys

In [2]:
# check pydov path
import pydov

## Get information about the datatype 'Formele stratigrafie'

In [3]:
from pydov.search.interpretaties import FormeleStratigrafieSearch
itp = FormeleStratigrafieSearch()

A description is provided for the 'Formele stratigrafie' datatype:

In [4]:
itp.get_description()

'Een formele stratigrafie van een boring of een sondering is een lithostratigrafische interpretatie van een (gecodeerde) lithologische beschrijving op basis van een welbepaalde vastgelegde standaard per diepte-interval. DOV gebruikt de gangbare standaarden binnen Vlaanderen. U vindt een overzicht van de gebruikte standaarden op onze website. De formele stratigrafie wordt gebruikt om de dikte van het Quartair te identificeren, en om de onderliggende pakketten Neogene, Paleogene en/of oudere lagen te onderscheiden en te identificeren.'

The different fields that are available for objects of the 'Formele stratigrafie' datatype can be requested with the get_fields() method:

In [5]:
fields = itp.get_fields()

# print available fields
for f in fields.values():
    print(f['name'])

pkey_interpretatie
Type_proef
Proefnummer
Proeffiche
x
y
start_interpretatie_mtaw
diepte_tot_m
gemeente
Auteurs
Datum
Opdrachten
betrouwbaarheid_interpretatie
Geldig_van
Geldig_tot
eerste_invoer
geom
diepte_laag_van
diepte_laag_tot
lid1
relatie_lid1_lid2
lid2
pkey_boring
pkey_sondering


You can get more information of a field by requesting it from the fields dictionary:

* *name*: name of the field
* *definition*: definition of this field
* *cost*: currently this is either 1 or 10, depending on the datasource of the field. It is an indication of the expected time it will take to retrieve this field in the output dataframe.
* *notnull*: whether the field is mandatory or not
* *type*: datatype of the values of this field
* *codelist*: optionally, a codelist that describes the possible values of this field

Alternatively, you can list all the fields and their details by inspecting the `get_fields()` output or the search instance itself in a notebook:

In [6]:
itp

## Example use cases

### Get 'Formele stratigrafie' in a bounding box

Get data for all the 'Formele stratigrafie' interpretations that are geographically located within the bounds of the specified box.

The coordinates are in the Belgian Lambert72 (EPSG:31370) coordinate system and are given in the order of lower left x, lower left y, upper right x, upper right y.

In [7]:
from pydov.util.location import Within, Box

df = itp.search(location=Within(Box(153145, 206930, 153150, 206935)))
df.head()

[000/001] .
[000/001] .


,pkey_interpretatie,pkey_boring,pkey_sondering,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,lid1,relatie_lid1_lid2,lid2
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,NaN,onbekend,153147.0,206931.0,14.12,0.0,3.00,Q,T,Q
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,NaN,onbekend,153147.0,206931.0,14.12,3.0,14.05,U,T,Bc


The dataframe contains one 'Formele stratigrafie' interpretation where two layers ('laag') were identified. The available data are flattened to represent unique attributes per row of the dataframe.

Using the *pkey_interpretatie* field one can request the details of this interpretation in a webbrowser:

In [8]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/2000-001087


Using the field's *codelist* information, we can translate the stratigraphical codes to human readable layers:

In [10]:
df['lid1_lb'] = df['lid1'].map(fields['lid1']['codelist'].get_definition)
df['lid2_lb'] = df['lid2'].map(fields['lid2']['codelist'].get_definition)
df[['pkey_interpretatie', 'diepte_laag_van', 'diepte_laag_tot', 'lid1', 'lid1_lb', 'relatie_lid1_lid2', 'lid2', 'lid2_lb']]

,pkey_interpretatie,diepte_laag_van,diepte_laag_tot,lid1,lid1_lb,relatie_lid1_lid2,lid2,lid2_lb
0,https://www.dov.vlaanderen.be/data/interpretat...,0.0,3.00,Q,Groep van Quartaire afzetting,T,Q,Groep van Quartaire afzetting
1,https://www.dov.vlaanderen.be/data/interpretat...,3.0,14.05,U,Onbekend,T,Bc,Formatie van Berchem


### Get 'Formele stratigrafie' with specific properties

Next to querying interpretations based on their geographic location within a bounding box, we can also search for interpretations matching a specific set of properties. For this we can build a query using a combination of the 'FormeleStratigrafie' fields and operators provided by the WFS protocol.

A list of possible operators can be found below:

In [11]:
[i for i,j in inspect.getmembers(sys.modules['owslib.fes2'], inspect.isclass) if 'Property' in i]

['PropertyIsBetween',
 'PropertyIsEqualTo',
 'PropertyIsGreaterThan',
 'PropertyIsGreaterThanOrEqualTo',
 'PropertyIsLessThan',
 'PropertyIsLessThanOrEqualTo',
 'PropertyIsLike',
 'PropertyIsNotEqualTo',
 'PropertyIsNull',
 'SortProperty']

In this example we build a query using the *PropertyIsGreaterThan* and *PropertyIsEqualTo* operators to find all interpretations that are at least 20 m deep, that are deemed appropriate for a range of 1 km from a defined point:

In [12]:
from owslib.fes2 import And, PropertyIsGreaterThan, PropertyIsEqualTo
from pydov.util.location import WithinDistance, Point

query = And([PropertyIsEqualTo(propertyname='Betrouwbaarheid',
                              literal='goed'),
            PropertyIsGreaterThan(propertyname='diepte_tot_m',
                                 literal='20'),
           ])
            
df = itp.search(query=query, 
                location=WithinDistance(Point(153145, 206930), 1000))

df.head()

[000/001] .
[000/008] ........


,pkey_interpretatie,pkey_boring,pkey_sondering,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,lid1,relatie_lid1_lid2,lid2
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1972...,NaN,goed,153906.0,207506.0,16.00,0.00,1.25,A,T,A
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1972...,NaN,goed,153906.0,207506.0,16.00,1.25,2.25,El,O,Gt
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1972...,NaN,goed,153906.0,207506.0,16.00,2.25,22.25,Bc,T,Bc
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1972...,NaN,goed,153906.0,207506.0,16.00,22.25,23.00,Bm,T,Bm
4,https://www.dov.vlaanderen.be/data/interpretat...,NaN,https://www.dov.vlaanderen.be/data/sondering/1...,goed,153898.0,207038.0,14.88,0.00,4.20,Q,T,Q


Once again we can use the *pkey_interpretatie* as a permanent link to the information of these interpretations:

In [13]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/2006-240083
https://www.dov.vlaanderen.be/data/interpretatie/2006-240086
https://www.dov.vlaanderen.be/data/interpretatie/2006-240080
https://www.dov.vlaanderen.be/data/interpretatie/2006-240153
https://www.dov.vlaanderen.be/data/interpretatie/2006-240085
https://www.dov.vlaanderen.be/data/interpretatie/2006-240082
https://www.dov.vlaanderen.be/data/interpretatie/2006-240081
https://www.dov.vlaanderen.be/data/interpretatie/2023-372783


### Get 'Formele stratigrafie' in a bounding box based on specific properties

We can combine a query on attributes with a query on geographic location to get the interpretations within a bounding box that have specific properties.

The following example requests the interpretations of boreholes only, within the given bounding box.

(Note that the datatype of the *literal* parameter should be a string, regardless of the datatype of this field in the output dataframe.)

In [14]:
from owslib.fes2 import PropertyIsEqualTo

query = PropertyIsEqualTo(
            propertyname='Type_proef',
            literal='Boring')

df = itp.search(
    location=Within(Box(153145, 206930, 154145, 207930)),
    query=query
    )

df.head()

[000/001] .
[000/052] ...........................c....................c.
[050/052] ..


,pkey_interpretatie,pkey_boring,pkey_sondering,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,lid1,relatie_lid1_lid2,lid2
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2024...,NaN,goed,153958.67,207152.54,15.71,0.0,0.5,A,T,A
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2024...,NaN,goed,153958.67,207152.54,15.71,0.5,3.0,QH,T,QP
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2024...,NaN,goed,153958.67,207152.54,15.71,3.0,5.0,Q,T,Q
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2024...,NaN,goed,153958.67,207152.54,15.71,5.0,7.0,Bc,T,Bc
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1969...,NaN,goed,153309.00,207396.00,16.27,0.0,2.3,El,O,Gt


We can look at one of the interpretations in a webbrowser using its *pkey_interpretatie*:

In [15]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/2000-023808
https://www.dov.vlaanderen.be/data/interpretatie/2000-006799
https://www.dov.vlaanderen.be/data/interpretatie/2000-007696
https://www.dov.vlaanderen.be/data/interpretatie/2000-007699
https://www.dov.vlaanderen.be/data/interpretatie/2023-372867
https://www.dov.vlaanderen.be/data/interpretatie/2023-372917
https://www.dov.vlaanderen.be/data/interpretatie/2023-372639
https://www.dov.vlaanderen.be/data/interpretatie/2000-023815
https://www.dov.vlaanderen.be/data/interpretatie/2000-023869
https://www.dov.vlaanderen.be/data/interpretatie/2023-372847
https://www.dov.vlaanderen.be/data/interpretatie/2023-372710
https://www.dov.vlaanderen.be/data/interpretatie/2000-007695
https://www.dov.vlaanderen.be/data/interpretatie/2023-371952
https://www.dov.vlaanderen.be/data/interpretatie/2023-372783
https://www.dov.vlaanderen.be/data/interpretatie/2023-372916
https://www.dov.vlaanderen.be/data/interpretatie/2023-372885
https://www.dov.vlaander

### Select 'Formele stratigrafie' in a municipality and return date

We can limit the columns in the output dataframe by specifying the *return_fields* parameter in our search.

In this example we query all the 'Formele stratigrafie' interpretations in a distance of 1 km and return their date:

In [16]:
df = itp.search(location=WithinDistance(Point(153145, 206930), 1000),
                return_fields=('Datum',))
df.head()

[000/001] .


,Datum
0,2024-02-14
1,2023-04-14
2,1999-03-01
3,2000-05-20
4,2023-04-14


In [17]:
df.describe()

,Datum
count,62
unique,10
top,2000-05-20
freq,23


### Get 'Formele stratigrafie' based on fields not available in the standard output dataframe

To keep the output dataframe size acceptable, not all available WFS fields are included in the standard output. However, one can use this information to select interpretations as illustrated below.

For example, make a selection of the interpretations in municipality the of Antwerp, before 1/1/1990:

!*remark: mind that the municipality attribute is merely an attribute that is defined by the person entering the data. It can be ok, empty, outdated or wrong*!

In [18]:
from owslib.fes2 import And, PropertyIsEqualTo, PropertyIsLessThan

query = And([PropertyIsEqualTo(propertyname='gemeente',
                               literal='Antwerpen'),
             PropertyIsLessThan(propertyname='Datum', 
                                 literal='1990-01-01')]
            )
df = itp.search(query=query,
                return_fields=('pkey_interpretatie', 'Datum'))
df.head()

[000/001] .


,pkey_interpretatie,Datum
0,https://www.dov.vlaanderen.be/data/interpretat...,1984-09-10


### Get 'Formele stratigrafie' data, returning fields not available in the standard output dataframe

As denoted in the previous example, not all available fields are available in the default output frame to keep its size limited. However, you can request any available field by including it in the *return_fields* parameter of the search:

In [19]:
query = PropertyIsEqualTo(
            propertyname='gemeente',
            literal='Leuven')

df = itp.search(query=query,
                return_fields=('pkey_interpretatie', 'pkey_boring', 'pkey_sondering',
                               'x', 'y', 'start_interpretatie_mtaw', 'gemeente', 'Auteurs', 'Proefnummer'))

df.head()

[000/001] .


,pkey_interpretatie,pkey_boring,pkey_sondering,x,y,start_interpretatie_mtaw,gemeente,Auteurs,Proefnummer
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1991...,NaN,174081.0,175579.0,19.0,Leuven,"Van Frausem, A. - Katholieke Universiteit Leuv...",kb32d89e-B337
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1987...,NaN,170640.0,172567.0,30.0,Leuven,"Van Frausem, A. - Katholieke Universiteit Leuv...",kb32d89e-B326
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1905...,NaN,173237.0,175185.0,20.0,Leuven,"Claes, S. - Katholieke Universiteit Leuven (KUL)",kb32d89e-B16
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1925...,NaN,173267.0,175245.0,20.0,Leuven,"Claes, S. - Katholieke Universiteit Leuven (KUL)",kb32d89e-B130
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/2016...,NaN,173425.0,174461.0,21.0,Leuven,"Van Frausem, A. - Katholieke Universiteit Leuv...",kb32d89e-B511


## Visualize results

Using Folium, we can display the results of our search on a map.

In [20]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Transformer

In [21]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)
    x2,y2 = transformer.transform(x1, y1)
    return x2, y2

df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [22]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['Proefnummer'][loc]).add_to(marker_cluster)
fmap
